In [18]:
from metadrive.envs.metadrive_env import MetaDriveEnv
from metadrive.policy.expert_policy import ExpertPolicy
from metadrive.examples.ppo_expert.torch_expert import torch_expert as expert

from IPython.display import Image, clear_output
from pprint import pprint

In [19]:

def run_scenario_n_times_buggy(n_scenarios=10, seed = 0, expert_driving=True)-> list:
    """
        Runs same scenario n time and collects the traces
    """
    
    traces = []
    try:
        env=MetaDriveEnv(config={"map":"C",
                                "num_scenarios": n_scenarios}
        )

        for rep in range(n_scenarios):

            obs, step_info = env.reset(seed)
            step_info['repetition'] = rep
            traces.append(step_info)
            print(f'{env.current_seed = }')
            while True:
                # get action from expert driving, or a dummy action
                action = expert(env.agent, deterministic=True) if expert_driving else [0, 0.33]
                obs, reward, tm, tr, step_info = env.step(action)
                step_info['repetition'] = rep
                traces.append(step_info)
                
                if tm or tr:
                    break

    finally:
        env.close()

    return traces



In [20]:

def run_scenario_n_times_correct(n_scenarios=10, seed = 0, expert_driving=True)-> list:
    """
        Runs same scenario n time and collects the traces
    """
    
    traces = []
    try:

        for rep in range(n_scenarios):
            env=MetaDriveEnv(config={"map":"C",
                                    "num_scenarios": n_scenarios}
            )

            obs, step_info = env.reset(seed)
            step_info['repetition'] = rep
            traces.append(step_info)
            print(f'{env.current_seed = }')
            while True:
                
                # get action from expert driving, or a dummy action
                action = expert(env.agent, deterministic=True) if expert_driving else [0, 0.33]
                obs, reward, tm, tr, step_info = env.step(action)
                step_info['repetition'] = rep
                traces.append(step_info)
                
                if tm or tr:
                    break

            env.close()
    finally:
        pass

    return traces



In [21]:
seed = 7

buggy_traces = run_scenario_n_times_buggy(seed=seed, expert_driving=True)
deterministic_traces = run_scenario_n_times_correct(seed=seed, expert_driving=True)
clear_output()

In [22]:
import pandas as pd

def are_traces_deterministic(traces) -> bool:
    df = pd.DataFrame(traces)

    # grouping by repetition to get a list of traces
    traces = df.groupby('repetition')

    # drop index and repetition ID to compare only step info later
    stripped_traces = [trace.reset_index(drop=True).drop('repetition', axis=1) for _, trace in traces]

    # iterate over each trace and check if it is equal to the first one
    are_equal_to_first_trace = [trace.equals(stripped_traces[0]) for trace in stripped_traces]

    #if all traces are equal to the first, then all traces are equal hence deterministic
    return all(are_equal_to_first_trace)

In [23]:
are_traces_deterministic(buggy_traces)

False

In [24]:
are_traces_deterministic(deterministic_traces)

True